In [1]:
### Import & setup
import sys
sys.path.append('C:\\Users\\szb37\\My Drive\\Projects\\Unmasking power\\UP codebase')
import src.power as power
import src.config as config
import pandas as pd
from scipy import stats

# ignore guess_conf

In [5]:
### Ignore guess confidence
df_pop = power.Helpers.get_pop_data()
df_pop  = df_pop .drop(columns=['conf','gmg_se','gmg_ciL','gmg_ciH'])

gmg_C = df_pop.loc[(df_pop.trt=='C'), 'gmg'].to_numpy()
gmg_T = df_pop.loc[(df_pop.trt=='T'), 'gmg'].to_numpy()

ci_C = stats.ttest_1samp(gmg_C, popmean=0).confidence_interval(confidence_level=0.95)
ci_T = stats.ttest_1samp(gmg_T, popmean=0).confidence_interval(confidence_level=0.95)

df_ignore_gc = pd.DataFrame({
    'scenario': ['POP', 'POP'],
    'trial': [0, 0],
    'trt': ['C', 'T'],
    'gmg': [round(gmg_C.mean(),2), round(gmg_T.mean(),2)],
    'ciL': [round(ci_C.low,2), round(ci_T.low,2)],
    'ciH': [round(ci_C.high,2), round(ci_T.high,2)],
    'moe': [round((ci_C.high-ci_C.low)/2,2), round((ci_T.high-ci_T.low)/2,2)],
})
df_ignore_gc


,scenario,trial,trt,gmg,ciL,ciH,moe
0,POP,0,C,4.27,1.83,6.72,2.44
1,POP,0,T,17.69,10.64,24.74,7.05


In [6]:
df_ignore_gc['n']= [df_pop.loc[df_pop.trt=='C'].shape[0], df_pop.loc[df_pop.trt=='T'].shape[0]]
df_ignore_gc = power.Stats.get_df_differenceCI(df_ignore_gc, col_value='gmg')
df_ignore_gc

,scenario,trial,gmg,ciL,ciH,moe
0,POP,0,13.41,5.8,21.03,7.61


# guess_conf-to-CI

In [4]:
df_pop = power.Helpers.get_pop_data()
df_gc2CI = power.Stats.get_df_combinedCI_fixed(df_pop, col_value='gmg', col_ciL='gmg_ciL', col_ciH='gmg_ciH',)
#df_gc2CI = df_gc2CI.drop(columns=['scenario', 'trial'])
df_gc2CI

,scenario,trial,trt,gmg,ciL,ciH,moe
0,POP,0,C,4.27,2.67,5.88,1.60
1,POP,0,T,17.69,13.06,22.32,4.63


In [5]:
df_gc2CI['n']= [df_pop.loc[df_pop.trt=='C'].shape[0], df_pop.loc[df_pop.trt=='T'].shape[0]]
df_gc2CI = power.Stats.get_df_differenceCI(df_gc2CI, col_value='gmg')
df_gc2CI

,scenario,trial,gmg,ciL,ciH,moe
0,POP,0,13.42,8.42,18.41,5.0


# guess_conf-to-weight

In [7]:
### Calculate POP CIs with guess_conf-to-weigths approach
df_pop = power.Helpers.get_pop_data()
df_gc2w = power.Stats.get_df_weighted_gmgs(df_pop, col_value='gmg', col_conf='conf')
# df_gc2w = df_gc2w.drop(columns=['scenario', 'trial'])
df_gc2w

,scenario,trial,trt,gmg,ciL,ciH,moe
0,POP,0,C,3.73,1.48,5.98,2.25
1,POP,0,T,18.02,10.32,25.71,7.70


In [8]:
df_gc2w['n']= [df_pop.loc[df_pop.trt=='C'].shape[0], df_pop.loc[df_pop.trt=='T'].shape[0]]
df_gc2w = power.Stats.get_df_differenceCI(df_gc2w, col_value='gmg')
df_gc2w

,scenario,trial,gmg,ciL,ciH,moe
0,POP,0,14.28,6.07,22.5,8.22
